## Entity linking on example data

In [1]:
from spacy import displacy

from src.ann_linker.dag import entities, aliases, nlp, nlp_with_linker, kb
from src.ann_linker.linker import AnnLinker

/Users/louis.guitton/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
kb = kb(entities(), aliases())

/Users/louis.guitton/.pyenv/versions/3.11.4/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
nlp = nlp_with_linker(nlp(), kb)

In [4]:
ruler = nlp.add_pipe('entity_ruler', before="ner")
patterns = [
    {"label": "SKILL", "pattern": alias}
    for alias in [a.alias for a in aliases()] + ['machine learn']
]
ruler.add_patterns(patterns)

In [5]:
doc = nlp("NLP is a subset of machine learn.")

In [6]:
doc.ents

(NLP, machine learn)

In [7]:
displacy.render(doc, style="ent")

In [8]:
for ent in doc.ents:
    print(ent.kb_id_)

a3



In [9]:
doc.ents[1]._.kb_candidates

[(Entity(entity_id='a1', name='Machine learning (ML)', description='Machine learning (ML) is the scientific study of algorithms and statistical models...', label=None),
  0.5660956501960754)]

## Load ICIJ dataset

In [11]:
import spacy
from spacy.tokens import DocBin
from spacy import displacy

from src.scraper import SPACY_MODEL

In [12]:
doc_bin = DocBin().from_disk(path="data/dataset.spacy")
len(doc_bin)

2

In [13]:
scrape_nlp: spacy.Language = spacy.load(SPACY_MODEL)
docs = list(doc_bin.get_docs(scrape_nlp.vocab))

In [14]:
displacy.render(docs[0], style="ent")

## Entity linking from Senzing data